<a href="https://colab.research.google.com/github/Shallom12/Autonomous-driving-of-Chungnam-Human-Resources-Development-Institute/blob/main/Autodriving_test(class%2Cdef).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚗 자율주행 개발을 위한 Python 핵심 문법 가이드

> 자율주행 시스템 개발에서 가장 많이 사용되는 Python 문법들을 실제 예시와 함께 정리한 가이드입니다.

## 📋 목차

- [반복문 (Loops)](#반복문-loops)
- [배열 처리 (Array Processing)](#배열-처리-array-processing)
- [함수 정의 (Function Definition)](#함수-정의-function-definition)
- [클래스 정의 (Class Definition)](#클래스-정의-class-definition)
- [조건문 활용 (Conditional Statements)](#조건문-활용-conditional-statements)
- [제어문 (Control Statements)](#제어문-control-statements)
- [예외 처리 (Exception Handling)](#예외-처리-exception-handling)

---

## 🔄 반복문 (Loops)

### 실시간 비디오 처리 - while 루프

실시간으로 카메라에서 영상을 받아와 객체를 탐지하는 메인 루프입니다.

```python
# 기본 비디오 처리 루프
while True:
    ret, frame = cap.read()          # 카메라에서 프레임 읽기
    if not ret:                      # 프레임 읽기 실패시 종료
        break
    
    results = model(frame)           # AI 모델로 객체 탐지
    cv2.imshow('frame', frame)       # 화면에 출력
    
    if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키 누르면 종료
        break
```

### 탐지 결과 처리 - for 루프

AI 모델이 탐지한 객체들을 하나씩 처리하는 방법들입니다.

```python
# 각 탐지 결과 순회
for result in results:
    boxes = result.boxes
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box.xyxy[0]    # 바운딩 박스 좌표
        conf = box.conf[0]              # 신뢰도
        cls = box.cls[0]                # 클래스 (차량, 사람 등)
```

```python
# 좌표 배열 순회 - 더 간단한 방법
for i, (x1, y1, x2, y2) in enumerate(boxes):
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
```

```python
# range로 인덱스 접근 - 조건부 처리할 때 유용
for i in range(len(boxes)):
    if conf[i] > 0.5:               # 신뢰도 50% 이상만 처리
        # 처리 로직
        pass
```

---

## 📊 배열 처리 (Array Processing)

### 기본 배열 조작

자율주행에서 탐지 결과나 트래킹 히스토리를 관리할 때 사용하는 배열 처리 방법들입니다.

```python
# 빈 리스트 생성
detections = []                      # 탐지 결과 저장
track_history = []                   # 객체 이동 경로 저장

# 배열에 추가
detections.append([x1, y1, x2, y2, conf, cls])  # 탐지 정보 추가
track_history.append((cx, cy))                   # 중심점 좌표 추가

# 배열 길이 제한 (최근 N개만 유지) - 메모리 효율성
if len(track_history) > 30:
    track_history.pop(0)             # 가장 오래된 데이터 제거

# 배열 슬라이싱 - 최근 데이터만 분석
recent_points = track_history[-10:]  # 최근 10개 포인트
```

---

## ⚙️ 함수 정의 (Function Definition)

### 차선 검출 함수

복잡한 이미지 처리 로직을 재사용 가능한 함수로 만드는 예시입니다.

```python
def detect_lane_lines(img, roi_vertices, canny_low, canny_high, hough_threshold):
    """
    차선 검출 함수
    
    Args:
        img: 입력 이미지
        roi_vertices: 관심 영역 좌표 [(x1,y1), (x2,y2), ...]
        canny_low: Canny 엣지 검출 하위 임계값
        canny_high: Canny 엣지 검출 상위 임계값
        hough_threshold: Hough 변환 임계값
    
    Returns:
        차선이 그려진 이미지
    """
    # 차선 검출 로직 구현
    pass
```

#### 🔍 함수 인자 상세 설명

1. **img**: 차선을 검출할 입력 이미지
2. **roi_vertices**: 관심 영역(ROI) 좌표들 - 도로 부분만 집중 분석
3. **canny_low**: Canny 엣지 검출의 하위 임계값 - 약한 엣지 감지
4. **canny_high**: Canny 엣지 검출의 상위 임계값 - 강한 엣지 감지
5. **hough_threshold**: Hough 변환 임계값 - 직선 검출 민감도

### 객체 필터링 함수

원하는 종류의 객체만 골라내는 실용적인 함수입니다.

```python
def filter_by_class(boxes, conf, cls, target_classes):
    """
    특정 클래스의 객체만 골라내는 필터 함수
    우리가 원하는 물체만 골라내는 역할을 합니다.
    """
    filtered_boxes = []              # 선택된 물체들의 위치 저장
    filtered_conf = []               # 선택된 물체들의 신뢰도 저장
    
    for i, c in enumerate(cls):      # 각 클래스를 하나씩 검사
        if c in target_classes:      # 원하는 클래스인지 확인
            filtered_boxes.append(boxes[i])
            filtered_conf.append(conf[i])
    
    return filtered_boxes, filtered_conf
```

### 유틸리티 함수들

```python
def calculate_center(x1, y1, x2, y2):
    """바운딩 박스의 중심점 계산"""
    return int((x1 + x2) / 2), int((y1 + y2) / 2)

def is_in_danger_zone(cx, cy, img_width, img_height):
    """객체가 위험 구역에 있는지 판단"""
    return (cy > img_height * 0.7 and
            cx > img_width * 0.3 and
            cx < img_width * 0.7)
```

---

## 🏗️ 클래스 정의 (Class Definition)

### 차량 추적 클래스

여러 프레임에 걸쳐 동일한 객체를 추적하는 클래스입니다.

```python
class VehicleTracker:
    def __init__(self):
        """초기화 - 추적 시스템 설정"""
        self.tracks = {}             # 추적 중인 객체들 {id: 위치정보}
        self.next_id = 0             # 다음에 할당할 ID
        self.max_disappeared = 10    # 몇 프레임 사라지면 추적 중단
    
    def update_tracks(self, detections):
        """새로운 탐지 결과로 추적 정보 업데이트"""
        for detection in detections:
            track_id = self.find_closest_track(detection)
            if track_id is None:
                self.create_new_track(detection)    # 새 객체 추적 시작
            else:
                self.update_existing_track(track_id, detection)  # 기존 추적 업데이트
```

### 자율주행 메인 클래스

전체 자율주행 시스템을 관리하는 메인 클래스입니다.

```python
class AutonomousDriving:
    def __init__(self):
        self.tracker = VehicleTracker()
        
    def process_frame(self, frame):
        """한 프레임을 처리하고 주행 결정을 내림"""
        detections = self.detect_objects(frame)      # 객체 탐지
        self.tracker.update_tracks(detections)       # 객체 추적
        decision = self.make_driving_decision(detections)  # 주행 결정
        return decision

    def make_driving_decision(self, detections):
        """탐지 결과를 바탕으로 주행 결정"""
        if self.obstacle_ahead(detections):
            return "brake"                           # 앞에 장애물 → 제동
        elif self.lane_change_needed(detections):
            return "change_lane"                     # 차선 변경 필요
        else:
            return "continue"                        # 직진 계속
```

---

## 🎯 조건문 활용 (Conditional Statements)

### 다중 조건 처리

객체 타입에 따라 다른 색상으로 표시하는 예시입니다.

```python
# 객체 타입별 처리
if cls == 0:           # person (사람)
    color = (0, 255, 0)     # 초록색
elif cls == 2:         # car (자동차)
    color = (255, 0, 0)     # 빨간색
elif cls == 3:         # motorcycle (오토바이)
    color = (0, 0, 255)     # 파란색
else:
    color = (128, 128, 128) # 회색 (기타)
```

### 영역별 처리

화면을 3등분하여 객체가 어느 영역에 있는지 판단합니다.

```python
# 영역별 처리
if center_x < img.shape[1] // 3:
    zone = "left"                    # 좌측 영역
elif center_x > img.shape[1] * 2 // 3:
    zone = "right"                   # 우측 영역
else:
    zone = "center"                  # 중앙 영역
```

### 복합 조건

여러 조건을 동시에 만족하는 중요한 객체를 선별합니다.

```python
# 복합 조건 - 중요한 객체 선별
if (conf > 0.7 and                          # 높은 신뢰도
    cls in [0, 2, 3] and                     # 특정 클래스 (사람, 차량, 오토바이)
    y2 > img.shape[0] * 0.5):               # 화면 하단 (가까운 거리)
    
    important_objects.append([x1, y1, x2, y2])  # 중요 객체로 분류
```

---

## 🎮 제어문 (Control Statements)

### continue - 다음 반복으로

조건에 맞지 않는 경우 현재 반복을 건너뛰고 다음으로 넘어갑니다.

```python
# 프레임 처리에서 continue 사용
while True:
    ret, frame = cap.read()
    if not ret:
        continue                    # 프레임 읽기 실패시 다음 프레임으로
    
    # 프레임이 너무 어두우면 건너뛰기
    if np.mean(frame) < 50:
        continue
    
    # 탐지 결과가 없으면 건너뛰기
    results = model(frame)
    if len(results[0].boxes) == 0:
        continue
    
    # 정상적인 처리
    process_detections(results)
```

```python
# 탐지 결과 필터링에서 continue 사용
for i, detection in enumerate(detections):
    confidence = detection.conf
    if confidence < 0.5:
        continue                    # 신뢰도가 낮으면 건너뛰기
    
    # 관심 없는 클래스 건너뛰기
    if detection.cls not in [0, 2, 3]:    # person, car, motorcycle만
        continue
    
    # 화면 밖 객체 건너뛰기
    if detection.x1 < 0 or detection.y1 < 0:
        continue
    
    process_valid_detection(detection)    # 유효한 탐지만 처리
```

### pass - 아무것도 하지 않음

나중에 구현할 함수나 조건문에서 임시로 사용합니다.

```python
# 미구현 함수 플레이스홀더
def emergency_brake():
    # TODO: 긴급 제동 로직 구현
    pass

def lane_change_left():
    # TODO: 좌측 차선 변경 로직
    pass

def calculate_safe_distance():
    # TODO: 안전거리 계산 로직
    pass
```

```python
# 조건부 처리에서 빈 블록
for detection in detections:
    if detection.cls == 0:         # person
        handle_pedestrian(detection)
    elif detection.cls == 2:       # car
        handle_vehicle(detection)
    elif detection.cls == 3:       # motorcycle
        handle_motorcycle(detection)
    else:
        pass                       # 다른 객체는 무시
```

---

## ⚠️ 예외 처리 (Exception Handling)

### 안전한 배열 접근

배열이 비어있는 경우를 대비한 안전한 처리 방법입니다.

```python
# 안전한 배열 접근
if len(boxes) > 0:
    for box in boxes:
        # 처리 로직
        pass
else:
    print("No detections")          # 탐지된 객체가 없음
```

### 센서 클래스에서의 예외 처리

```python
class LidarSensor(BaseSensor):
    def read_data(self):
        """라이다 데이터 읽기"""
        return self.get_lidar_data()
    
    def calibrate(self):
        """라이다 캘리브레이션"""
        pass                        # 라이다는 자동 캘리브레이션
```

---

## 💡 실전 팁

### 성능 최적화
- 반복문에서 불필요한 계산을 피하세요
- 배열 크기를 제한하여 메모리 사용량을 관리하세요
- 조건문에서 가장 가능성이 높은 조건을 먼저 배치하세요

### 코드 가독성
- 함수와 변수에 의미있는 이름을 사용하세요
- 복잡한 조건문은 함수로 분리하세요
- 주석을 활용하여 로직을 설명하세요

### 디버깅
- `print()` 문을 활용하여 값을 확인하세요
- 단계별로 결과를 시각화하세요
- 예외 상황을 미리 고려하여 방어적으로 코딩하세요

---

## 📚 참고 자료

- [OpenCV 공식 문서](https://opencv.org/)
- [YOLO 객체 탐지](https://github.com/ultralytics/yolov5)
- [Python 공식 문서](https://docs.python.org/3/)

---

*이 가이드가 자율주행 개발에 도움이 되었다면 ⭐ 스타를 눌러주세요!*


In [29]:
# 1. 기본적인 자동차 클래스: 자율주행차의 가장 기본이 되는 객체입니다.
class Car:
    def __init__(self, brand, model, color):
        # __init__은 클래스의 객체가 생성될 때 자동으로 호출되는 메서드입니다.
        # self는 현재 생성되는 객체 자신을 의미하며, 각 속성(brand, model, color)을 초기화합니다.
        self.brand = brand
        self.model = model
        self.color = color
        self.speed = 0 # 초기 속도는 0으로 설정합니다.

    def accelerate(self, increment):
        # accelerate 메서드는 자동차의 속도를 증가시킵니다.
        self.speed += increment
        print(f"{self.brand} {self.model}의 속도가 {self.speed} km/h로 증가했습니다.")

    def brake(self, decrement):
        # brake 메서드는 자동차의 속도를 감소시킵니다.
        self.speed = max(0, self.speed - decrement) # 속도는 0 미만이 될 수 없습니다.
        print(f"{self.brand} {self.model}의 속도가 {self.speed} km/h로 감소했습니다.")

# 자동차 객체 생성 및 사용 예시
my_car = Car("Hyundai", "IONIQ 5", "Silver")
my_car.accelerate(50)
my_car.brake(20)

print("-" * 30)



Hyundai IONIQ 5의 속도가 50 km/h로 증가했습니다.
Hyundai IONIQ 5의 속도가 30 km/h로 감소했습니다.
------------------------------


In [30]:
# 2. 센서 클래스: 자율주행차의 눈과 귀 역할을 하는 센서를 표현합니다.
class Sensor:
    def __init__(self, sensor_type, range_m):
        # 센서의 종류(LiDAR, Camera 등)와 측정 가능 거리(range_m)를 초기화합니다.
        self.sensor_type = sensor_type
        self.range_m = range_m
        self.data = None # 센서에서 얻은 데이터를 저장할 변수입니다.

    def detect_obstacle(self):
        # 장애물을 감지하는 시뮬레이션 메서드입니다. 실제로는 복잡한 로직이 필요합니다.
        import random
        # 0 또는 1을 무작위로 반환하여 장애물 감지 여부를 시뮬레이션합니다.
        self.data = random.choice([True, False])
        if self.data:
            print(f"{self.sensor_type} 센서가 {self.range_m}m 이내의 장애물을 감지했습니다.")
        else:
            print(f"{self.sensor_type} 센서에 장애물이 감지되지 않았습니다.")
        return self.data

# 센서 객체 생성 및 사용 예시
lidar_sensor = Sensor("LiDAR", 150)
camera_sensor = Sensor("Camera", 80)

lidar_sensor.detect_obstacle()
camera_sensor.detect_obstacle()

print("-" * 30)


LiDAR 센서가 150m 이내의 장애물을 감지했습니다.
Camera 센서에 장애물이 감지되지 않았습니다.
------------------------------


In [31]:

# 3. 경로 계획 클래스: 자율주행차가 목적지까지 어떻게 이동할지 계획합니다.
class PathPlanner:
    def __init__(self, start_location, end_location):
        # 시작 위치와 목적지 위치를 초기화합니다.
        self.start_location = start_location
        self.end_location = end_location
        self.path = [] # 계획된 경로를 저장할 리스트입니다.

    def plan_route(self):
        # 간단한 경로 계획 시뮬레이션입니다. 실제로는 복잡한 알고리즘이 사용됩니다.
        print(f"경로를 계획 중입니다: {self.start_location}에서 {self.end_location}까지")
        # 예시 경로를 추가합니다.
        self.path = [self.start_location, "Intersection A", "Intersection B", self.end_location]
        print(f"경로가 계획되었습니다: {', '.join(self.path)}")
        return self.path

    def update_route(self, current_location, new_destination=None):
        # 경로를 업데이트하는 메서드입니다. 교통 상황 등에 따라 경로가 바뀔 수 있습니다.
        print(f"현재 위치({current_location})에서 경로를 업데이트합니다.")
        if new_destination:
            self.end_location = new_destination
            print(f"새로운 목적지: {self.end_location}")
        # 여기서는 단순히 다시 계획하는 것으로 시뮬레이션합니다.
        self.plan_route()

# 경로 계획 객체 생성 및 사용 예시
planner = PathPlanner("Seoul Station", "Gangnam Station")
planner.plan_route()
planner.update_route("Intersection A", "Yeouido Park")

print("-" * 30)



경로를 계획 중입니다: Seoul Station에서 Gangnam Station까지
경로가 계획되었습니다: Seoul Station, Intersection A, Intersection B, Gangnam Station
현재 위치(Intersection A)에서 경로를 업데이트합니다.
새로운 목적지: Yeouido Park
경로를 계획 중입니다: Seoul Station에서 Yeouido Park까지
경로가 계획되었습니다: Seoul Station, Intersection A, Intersection B, Yeouido Park
------------------------------


In [32]:
# 4. 교통 신호등 클래스: 자율주행차가 도로의 규칙을 이해하고 따르도록 돕습니다.
class TrafficLight:
    def __init__(self, location, current_state="red"):
        # 신호등의 위치와 현재 상태(빨간불, 초록불, 노란불)를 초기화합니다.
        self.location = location
        self.current_state = current_state

    def change_state(self, new_state):
        # 신호등의 상태를 변경하는 메서드입니다.
        if new_state in ["red", "green", "yellow"]:
            self.current_state = new_state
            print(f"{self.location} 신호등이 {self.current_state}로 변경되었습니다.")
        else:
            print("유효하지 않은 신호등 상태입니다. (red, green, yellow 중 선택)")

    def get_state(self):
        # 현재 신호등 상태를 반환합니다.
        return self.current_state

# 교통 신호등 객체 생성 및 사용 예시
main_intersection_light = TrafficLight("Main Intersection")
print(f"현재 신호등 상태: {main_intersection_light.get_state()}")
main_intersection_light.change_state("green")
main_intersection_light.change_state("yellow")

print("-" * 30)



현재 신호등 상태: red
Main Intersection 신호등이 green로 변경되었습니다.
Main Intersection 신호등이 yellow로 변경되었습니다.
------------------------------


In [33]:
# 5. 자율주행 시스템 통합 클래스: 위에서 만든 클래스들을 통합하여 하나의 자율주행 시스템을 구성합니다.
class AutonomousVehicleSystem:
    def __init__(self, car, lidar_sensor, camera_sensor, path_planner):
        # 자율주행 시스템을 구성하는 핵심 구성 요소들을 초기화합니다.
        self.car = car
        self.lidar_sensor = lidar_sensor
        self.camera_sensor = camera_sensor
        self.path_planner = path_planner
        self.current_speed = 0
        self.target_speed = 0

    def start_driving(self):
        # 자율주행을 시작하는 시뮬레이션 메서드입니다.
        print(f"\n{self.car.brand} {self.car.model} 자율주행 시스템을 시작합니다.")
        self.path_planner.plan_route() # 경로 계획

    def update_driving_status(self):
        # 주기적으로 주행 상태를 업데이트하는 메서드입니다.
        print("\n주행 상태 업데이트 중...")
        # 센서 데이터 수집
        obstacle_detected_lidar = self.lidar_sensor.detect_obstacle()
        obstacle_detected_camera = self.camera_sensor.detect_obstacle()

        # 경로 추적 및 속도 제어 (간단한 로직)
        if obstacle_detected_lidar or obstacle_detected_camera:
            self.target_speed = 0 # 장애물 감지 시 정지
            print("장애물 감지! 속도를 0으로 줄입니다.")
        else:
            self.target_speed = 60 # 장애물 없을 시 목표 속도 설정
            print("경로를 따라 주행 중... 목표 속도: 60 km/h")

        # 실제 차량 속도 조절
        if self.car.speed < self.target_speed:
            self.car.accelerate(self.target_speed - self.car.speed)
        elif self.car.speed > self.target_speed:
            self.car.brake(self.car.speed - self.target_speed)

# 자율주행 시스템 객체 생성 및 사용 예시
my_car_for_system = Car("Tesla", "Model Y", "White")
lidar = Sensor("LiDAR", 200)
camera = Sensor("Camera", 100)
planner_for_system = PathPlanner("Home", "Office")

autonomous_system = AutonomousVehicleSystem(my_car_for_system, lidar, camera, planner_for_system)
autonomous_system.start_driving()
autonomous_system.update_driving_status()
autonomous_system.update_driving_status() # 여러 번 호출하여 상태 변화 확인

print("-" * 30)


Tesla Model Y 자율주행 시스템을 시작합니다.
경로를 계획 중입니다: Home에서 Office까지
경로가 계획되었습니다: Home, Intersection A, Intersection B, Office

주행 상태 업데이트 중...
LiDAR 센서에 장애물이 감지되지 않았습니다.
Camera 센서에 장애물이 감지되지 않았습니다.
경로를 따라 주행 중... 목표 속도: 60 km/h
Tesla Model Y의 속도가 60 km/h로 증가했습니다.

주행 상태 업데이트 중...
LiDAR 센서에 장애물이 감지되지 않았습니다.
Camera 센서가 100m 이내의 장애물을 감지했습니다.
장애물 감지! 속도를 0으로 줄입니다.
Tesla Model Y의 속도가 0 km/h로 감소했습니다.
------------------------------


In [34]:
# 1. 속도 계산 함수: 시간에 따른 이동 거리를 기반으로 속도를 계산합니다.
def calculate_speed(distance_m, time_s):
    # 속도 = 거리 / 시간 공식을 사용합니다.
    # 시간(s)이 0인 경우 오류를 방지합니다.
    if time_s == 0:
        return 0
    speed_mps = distance_m / time_s # 미터/초 (m/s)
    speed_kph = speed_mps * 3.6 # 미터/초를 킬로미터/시로 변환 (1 m/s = 3.6 km/h)
    print(f"이동 거리: {distance_m}m, 소요 시간: {time_s}초")
    print(f"계산된 속도: {speed_kph:.2f} km/h")
    return speed_kph

# 함수 사용 예시
speed1 = calculate_speed(100, 5)
speed2 = calculate_speed(500, 10)

print("-" * 30)


이동 거리: 100m, 소요 시간: 5초
계산된 속도: 72.00 km/h
이동 거리: 500m, 소요 시간: 10초
계산된 속도: 180.00 km/h
------------------------------


In [35]:
# 2. 장애물 거리 측정 함수: 센서 데이터(가상의)를 바탕으로 장애물까지의 거리를 반환합니다.
def measure_obstacle_distance(sensor_data):
    # sensor_data는 장애물까지의 거리를 나타내는 숫자라고 가정합니다.
    if sensor_data is None:
        print("센서 데이터가 유효하지 않습니다.")
        return None
    distance = float(sensor_data)
    print(f"감지된 장애물까지의 거리: {distance:.2f} 미터")
    return distance

# 함수 사용 예시
obstacle_dist1 = measure_obstacle_distance(15.7)
obstacle_dist2 = measure_obstacle_distance(5.1)
obstacle_dist3 = measure_obstacle_distance(None)

print("-" * 30)


감지된 장애물까지의 거리: 15.70 미터
감지된 장애물까지의 거리: 5.10 미터
센서 데이터가 유효하지 않습니다.
------------------------------


In [36]:
# 3. 차선 인식 함수: 이미지 데이터(가상의)를 분석하여 차선이 있는지 확인합니다.
def detect_lane(image_data):
    # image_data는 이미지 처리 결과로 차선이 감지되었는지 여부를 나타내는 불리언 값이라고 가정합니다.
    if image_data: # True일 경우 차선이 감지되었다고 가정
        print("차선이 성공적으로 감지되었습니다.")
        return True
    else:
        print("차선이 감지되지 않았습니다.")
        return False

# 함수 사용 예시
lane_status1 = detect_lane(True) # 차선이 있다고 가정
lane_status2 = detect_lane(False) # 차선이 없다고 가정

print("-" * 30)


차선이 성공적으로 감지되었습니다.
차선이 감지되지 않았습니다.
------------------------------


In [37]:
# 4. 주행 모드 변경 함수: 자율주행 모드를 수동, 자율, 반자율 등으로 변경합니다.
def set_driving_mode(mode):
    # 가능한 주행 모드를 정의합니다.
    valid_modes = ["manual", "autonomous", "semi-autonomous"]
    if mode in valid_modes:
        print(f"주행 모드가 '{mode}'로 변경되었습니다.")
        return True
    else:
        print(f"'{mode}'는 유효하지 않은 주행 모드입니다. 가능한 모드: {', '.join(valid_modes)}")
        return False

# 함수 사용 예시
set_driving_mode("autonomous")
set_driving_mode("manual")
set_driving_mode("sport") # 유효하지 않은 모드

print("-" * 30)


주행 모드가 'autonomous'로 변경되었습니다.
주행 모드가 'manual'로 변경되었습니다.
'sport'는 유효하지 않은 주행 모드입니다. 가능한 모드: manual, autonomous, semi-autonomous
------------------------------


In [38]:
# 5. 비상 정지 함수: 위험 상황 발생 시 차량을 즉시 정지시킵니다.
def emergency_stop(current_speed):
    if current_speed > 0:
        print(f"위험 상황 감지! 현재 속도 {current_speed} km/h에서 비상 정지합니다.")
        # 실제로는 차량의 브레이크 시스템을 작동시키는 코드가 여기에 들어갑니다.
        final_speed = 0
        print("차량이 완전히 정지했습니다.")
        return final_speed
    else:
        print("차량이 이미 정지 상태입니다. 비상 정지할 필요가 없습니다.")
        return 0

# 함수 사용 예시
car_speed_now = 80
emergency_stop(car_speed_now)
emergency_stop(0)

print("-" * 30)

위험 상황 감지! 현재 속도 80 km/h에서 비상 정지합니다.
차량이 완전히 정지했습니다.
차량이 이미 정지 상태입니다. 비상 정지할 필요가 없습니다.
------------------------------


In [39]:
#!/usr/bin/env python
# coding: utf-8

# # [미션] 서울 아파트 가격 데이터 분석

# 이번 미션에서는 2022년 서울 아파트 거래 데이터를 분석해보도록 하겠습니다.

# In[1]:


import pandas as pd
import numpy as np


# 데이터셋을 불러오고 확인합니다.

# In[2]:


# seoul_apart_2022.csv 파일을 읽어와 DataFrame 객체 df를 생성합니다.
# 이 파일은 2022년 서울 아파트 실거래가 데이터를 담고 있습니다.
df=pd.read_csv("./sample_data/seoul_apart_2022.csv")

# head() 함수를 사용하여 데이터의 첫 5개 행을 출력하여 구조를 확인합니다.
df.head()


# 전체 데이터의 갯수와 자료형 등을 확인합니다.

# In[3]:


# info() 함수를 사용하여 데이터프레임의 전체적인 정보를 확인합니다.
# 각 컬럼의 이름, null이 아닌 값의 개수, 데이터 타입을 보여줍니다.
df.info()


# ### [TODO] 데이터프레임 `df`에서 사용하지 않을 특정 컬럼을 제거하세요.
#
# * `drop`을 활용해서 데이터 분석 과정에서 활용하기 어려운 컬럼들을 삭제합니다.
# * 삭제할 컬럼은 다음과 같습니다: **"해제사유발생일", "중개사소재지", "번지", "본번", "부번", "도로명", "거래유형"**
# * 컬럼 삭제가 데이터프레임 `df`에 적용이 되어야 합니다.

# In[4]:


# [TODO] 1: 불필요한 컬럼 삭제
# drop 함수를 사용하여 지정된 컬럼들을 제거합니다.
# axis=1은 열(column)을 기준으로 삭제하겠다는 의미입니다.
# 원본 데이터프레임 'df'에 변경 사항을 바로 적용하기 위해, 결과를 다시 'df'에 할당합니다.
df = df.drop(["해제사유발생일", "중개사소재지", "번지", "본번", "부번", "도로명", "거래유형"], axis =1)


# "전용면적(㎡)" 컬럼의 제곱미터가 특수문자라 사용하기 어려움으로 일단 "전용면적"으로 바꾸도록 하겠습니다.

# In[5]:


# rename 함수를 사용하여 '전용면적(㎡)' 컬럼의 이름을 '전용면적'으로 변경합니다.
# columns 딕셔너리에 {기존이름:새이름} 형식으로 지정합니다.
# 원본 데이터프레임 'df'에 변경 사항을 적용하기 위해 결과를 다시 'df'에 할당합니다.
df=df.rename(columns={"전용면적(㎡)":"전용면적"})


# "시군구"컬럼의 주소는 활용하기 쉽게 "구" 컬럼과 "동" 컬럼으로 분리합니다.

# In[6]:


# '시군구' 컬럼에서 '구'와 '동' 정보를 추출하여 새로운 컬럼을 만듭니다.
# apply와 lambda 함수를 사용하여 각 행의 '시군구' 문자열을 공백으로 분리합니다.
# e.g., "서울특별시 강남구 개포동" -> ['서울특별시', '강남구', '개포동']
df["구"]=df["시군구"].apply(lambda e: e.split()[1]) # 분리된 리스트의 두 번째 요소([1])가 '구'입니다.
df["동"]=df["시군구"].apply(lambda e: e.split()[2]) # 분리된 리스트의 세 번째 요소([2])가 '동'입니다.
df.head()


# 전용면적이 60 이하면 **소형**, 60보다 크고 85 이하면 **중형**, 85보다 크고 102 이하면 **중대형**, 102보다 크면 **대형**으로 분류됩니다.

# In[7]:


# 전용면적 크기에 따라 아파트 유형을 분류하는 함수를 정의합니다.
def category(e):
    if e<=60:
        return "소형"
    elif e<=85:
        return "중형"
    elif e<=102:
        return "중대형"
    else:
        return "대형"

# apply 함수를 사용하여 '전용면적' 컬럼의 모든 값에 category 함수를 적용합니다.
# 그 결과를 '유형'이라는 새로운 컬럼에 저장합니다.
df["유형"]=df["전용면적"].apply(category)
df.head()


# "계약년월" 컬럼과 "계약일" 컬럼을 합치고, 날짜타입으로 변경합니다.

# In[8]:


# '계약년월'과 '계약일'을 문자열로 변환하여 합친 후, '계약일' 컬럼에 다시 저장합니다.
df["계약일"]=df["계약년월"].astype('str') + df["계약일"].astype(str)
# pd.to_datetime 함수를 사용하여 합쳐진 '계약일' 문자열을 datetime 객체로 변환합니다.
# format='%Y%m%d'는 '20220101'과 같은 형식의 문자열을 파싱하는 규칙입니다.
df["계약일"] = pd.to_datetime(df["계약일"], format='%Y%m%d')

df["계약일"].info()


# "계약일" 컬럼의 데이터가 `datetime`으로 변경되었습니다. 이제 계약일 데이터를 활용해 "계약월"과 "계약요일" 컬럼을 생성해보겠습니다.

# In[9]:


# datetime으로 변환된 '계약일' 컬럼에서 월과 요일 정보를 추출합니다.
# .dt 접근자를 사용하여 datetime 속성에 접근할 수 있습니다.
df["계약월"]=df["계약일"].dt.month # 월 정보 추출
df["계약요일"]=df["계약일"].dt.dayofweek # 요일 정보 추출 (월요일=0, 일요일=6)
# map 함수를 사용하여 숫자 요일을 문자열(월, 화, 수...)로 변환합니다.
df["계약요일"]=df["계약요일"].map({0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'})

df.head()


# 부동산 거래 데이터에서 가장 중요한 데이터는 "거래금액(만원)"입니다. 이 거래금액을 활용하여 정렬을 하거나 다양한 통계값을 계산할 수 있습니다.
#
# "거래금액(만원)" 컬럼을 활용하기 전에 데이터의 결측치를 확인해보도록 하겠습니다.

# In[10]:


# isnull() 함수로 각 데이터가 결측치(NaN)인지 확인하고, sum() 함수로 컬럼별 결측치 개수를 합산합니다.
df.isnull().sum()


# "거래금액(만원)" 컬럼에 33개의 결측치가 있습니다. 거래금액은 데이터 분석에서 굉장히 중요한 정보이기 때문에 거래금액 정보가 없는 데이터는 사용할 수 없고, 다른 통계값으로 채워넣기에도 좋지 않습니다.
#
# 마침 데이터가 12000개가 넘어서 꽤 많기 때문에, 이번에는 "거래금액(만원)"컬럼에 결측치가 있는 데이터는 아예 삭제해버리도록 하겠습니다.
#
# ### [TODO] 데이터프레임 `df`에서 "거래금액(만원)"컬럼에 결측치가 존재하는 데이터를 삭제하세요.
#
# * `dropna`를 활용해서 "거래금액(만원)" 컬럼에 결측치가 존재하는 아파트 거래 데이터를 삭제하세요.
# * 데이터 삭제 후 `dropna`의 `ignore_index`를 올바르게 설정하여 인덱스를 초기화해주어야 합니다.

# In[11]:


# [TODO] 2: 결측치 데이터 삭제
# dropna 함수를 사용하여 결측치가 있는 행을 삭제합니다.
# subset=['거래금액(만원)'] 인자를 통해 '거래금액(만원)' 컬럼에 결측치가 있는 행만 대상으로 지정합니다.
# ignore_index=True는 행이 삭제된 후 인덱스를 0부터 다시 시작하도록 재설정합니다.
# 원본 데이터프레임 'df'에 변경 사항을 적용하기 위해, 결과를 다시 'df'에 할당합니다.
df = df.dropna(subset=['거래금액(만원)'], ignore_index = True)


# In[12]:


# 결측치 제거 후 데이터프레임 정보를 다시 확인합니다.
# '거래금액(만원)' 컬럼의 Non-Null Count가 전체 Entries 수와 같아진 것을 볼 수 있습니다.
df.info()


# 데이터가 33개 줄어들기는 했지만, 결측치가 있는 데이터가 삭제되었습니다.
#
# `info`를 통해 데이터의 정보를 다시 확인해보면 "거래금액(만원)"컬럼의 경우 데이터타입이 `object`인 것을 확인할 수 있습니다. 원활한 데이터 분석을 위해 콤마(,)를 삭제하고 숫자형태 데이터로 바꾸어주도록 하겠습니다.

# In[13]:


# '거래금액(만원)' 컬럼의 데이터 타입을 숫자로 변환합니다.
# .str.replace(",", "")를 사용하여 금액에 포함된 쉼표(,)를 제거합니다.
df["거래금액(만원)"]=df["거래금액(만원)"].str.replace(",","")
# pd.to_numeric 함수를 사용하여 문자열 타입의 숫자를 실제 숫자(정수 또는 실수) 타입으로 변환합니다.
df["거래금액(만원)"] = pd.to_numeric(df["거래금액(만원)"])

# info() 함수로 해당 컬럼의 데이터 타입이 변경되었는지 확인합니다.
df["거래금액(만원)"].info()


# "거래금액(만원)" 컬럼의 데이터 타입이 정수형으로 바뀐 것을 확인할 수 있습니다. 이제 거래금액을 기준으로 내림차순으로 정렬해보도록 하겠습니다.

# In[14]:


# sort_values 함수를 사용하여 '거래금액(만원)'을 기준으로 데이터를 정렬합니다.
# ascending=False는 내림차순(큰 값이 먼저 오도록)으로 정렬하라는 의미입니다.
# head(10)으로 상위 10개의 가장 비싼 거래를 확인합니다.
df.sort_values("거래금액(만원)", ascending=False).head(10)


# 이번엔 "거래금액(만원)" 컬럼의 평균값과 중앙값을 계산해보겠습니다.

# In[15]:


# mean() 함수로 '거래금액(만원)'의 산술 평균을 계산하고 출력합니다.
print("거래금액 평균값(만원): ", df["거래금액(만원)"].mean())
# median() 함수로 '거래금액(만원)'의 중앙값(데이터를 순서대로 나열했을 때 가운데 값)을 계산하고 출력합니다.
print("거래금액 중앙값(만원): ", df["거래금액(만원)"].median())


# 아파트의 거래금액 자체도 중요하지만 크기에 따른 가격 또한 매우 중요한 지표입니다.
#
# "전용면적"컬럼의 단위는 제곱미터로, 이는 국제 표준 규격이지만 우리에게는 평 단위가 조금 더 익숙합니다. 1평은 대략 3.3㎡ 이므로, "전용면적"컬럼의 값을 3.3으로 나누어 덮어씌우고 컬럼의 이름을 "전용면적(평)"으로 변경합니다.

# In[16]:


# '전용면적'을 제곱미터(㎡)에서 평 단위로 변환합니다. (1평 ≈ 3.3㎡)
# 계산된 결과를 round 함수를 사용해 소수점 둘째 자리에서 반올림합니다.
df["전용면적"] = round(df["전용면적"]/3.3, 2)
# 컬럼 이름을 '전용면적(평)'으로 변경하여 단위를 명확히 합니다.
df=df.rename(columns={"전용면적":"전용면적(평)"})

df.head()


# 이제 "전용면적(평)"과 "거래금액(만원)" 컬럼을 활용해 "평당금액" 컬럼을 생성해보도록 하겠습니다.

# ### [TODO] 데이터프레임 `df`에 "평당금액" 컬럼을 생성하세요.
#
# * 시리즈 연산을 활용한 데이터 변환을 통해 `df`에 "평당금액" 컬럼을 생성하세요.
# * 평당금액은 **"거래금액(만원)"값을 "전용면적(평)"값으로 나눈 값**입니다.

# In[17]:


# [TODO] 3: '평당금액' 컬럼 생성
# '거래금액(만원)' 컬럼의 값을 '전용면적(평)' 컬럼의 값으로 나누어 평당 가격을 계산합니다.
# 이 결과를 '평당금액'이라는 새로운 컬럼에 저장합니다.
df['평당금액'] = df['거래금액(만원)']/df['전용면적(평)']


# In[18]:


# '평당금액' 컬럼의 값을 소수점 둘째 자리에서 반올림하여 가독성을 높입니다.
df["평당금액"]=round(df["평당금액"],2)
# info() 함수를 통해 '평당금액' 컬럼이 정상적으로 추가되었는지 확인합니다.
df.info()


# 이제 평당금액을 기준으로 다시 내림차순 정렬해보도록 하겠습니다.

# In[19]:


# sort_values를 사용하여 '평당금액'을 기준으로 내림차순 정렬하고 상위 10개를 확인합니다.
# 이를 통해 단위 면적당 가격이 가장 비싼 아파트를 볼 수 있습니다.
df.sort_values("평당금액", ascending=False).head(10)


# 앞서 거래금액을 기준으로 정렬했던결과 많이 다른 결과를 얻을 수 있습니다. 이러한 결과를 부동산 도메인 지식과 함께 활용한다면 유용한 분석이 이루어질 수 있을 것입니다. 이렇게 새로운 지표로 이루어진 컬럼을 생성하는 것은 데이터 분석 과정에서 매우 중요합니다.
#
# 이번엔 "구"컬럼을 기준으로 묶어 서울의 각 구별 평당금액의 평균값을 확인해보도록 하겠습니다.

# In[20]:


# groupby("구")를 사용하여 데이터를 '구'별로 그룹화합니다.
# [['평당금액']]으로 평당금액 컬럼을 선택하고, mean() 함수로 각 구의 평당금액 평균을 계산합니다.
df.groupby("구")[["평당금액"]].mean()


# ### [TODO] 서초구의 동 중 **평당금액** 평균이 2번째로 높은 동의 이름을 입력하세요.
# * `groupby`와 "구", "동" 컬럼을 활용하여 서초구의 각 동별 **평당금액 평균값**을 구할 수 있습니다.
# * `groupby`를 통해 출력된 결과를 확인하고 평당금액 평균값이 2번째로 높은 동의 이름을 문자열 형태로 변수 `ans1`에 저장하세요.
# * ex) 정답이 망원동일 경우 -> `ans1 = "망원동"`

# In[24]:


# [TODO] 4: 서초구 내 동별 평당금액 순위 분석
# 1. '구' 컬럼이 '서초구'인 데이터만 필터링합니다.
seocho_df = df[df['구'] == '서초구']

# 2. 필터링된 데이터를 '동'으로 그룹화하고, 각 동의 '평당금액' 평균을 계산합니다.
# 3. sort_values를 이용해 계산된 평균값을 기준으로 내림차순 정렬합니다.
dong_rank = seocho_df.groupby('동')[['평당금액']].mean().sort_values('평당금액', ascending=False)

# 4. 정렬된 결과에서 두 번째 행의 인덱스(동 이름)를 가져옵니다. (인덱스는 0부터 시작하므로 1이 두 번째입니다.)
#    dong_rank.index는 동 이름들을 담고 있는 인덱스 객체입니다.
ans1 = dong_rank.index[1]

# dong_rank를 출력하여 순위를 직접 확인할 수 있습니다.
# 1위는 잠원동, 2위는 반포동인 것을 알 수 있습니다.
print(dong_rank)


# In[22]:


# [정답 제출]
# 위 코드에서 계산된 ans1 변수에는 '반포동'이 저장됩니다.
ans1 = "반포동"


# ## 채점
# * **[TODO]** 중 수행하지 않은 부분이 없는지 확인하세요.
# * 채점을 위해 아래 코드를 실행한 뒤 우측 상단의 제출 버튼을 눌러주세요.
# * 코드 수정시 정상적인 채점이 이루어지지 않습니다.

# In[23]:


import os
import json

df.to_json("problem_2.json", force_ascii=False)

result = {}
result["problem_1"] = str(ans1)
result["problem_2"] = "problem_2.json"

with open("result.json", "w") as f:
    json.dump(result, f)


# In[ ]:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군구       12684 non-null  object 
 1   번지        12681 non-null  object 
 2   본번        12682 non-null  float64
 3   부번        12682 non-null  float64
 4   단지명       12684 non-null  object 
 5   전용면적(㎡)   12684 non-null  float64
 6   계약년월      12684 non-null  int64  
 7   계약일       12684 non-null  int64  
 8   거래금액(만원)  12651 non-null  object 
 9   층         12684 non-null  int64  
 10  건축년도      12682 non-null  float64
 11  도로명       12684 non-null  object 
 12  해제사유발생일   715 non-null    float64
 13  거래유형      12684 non-null  object 
 14  중개사소재지    12684 non-null  object 
dtypes: float64(5), int64(3), object(7)
memory usage: 1.5+ MB
<class 'pandas.core.series.Series'>
RangeIndex: 12684 entries, 0 to 12683
Series name: 계약일
Non-Null Count  Dtype         
--------------  -----         
12684 non-nul